In [18]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [19]:
with open('tweetspakvsind.txt',encoding="utf-8") as fp:
    plaintext=fp.read()
    plaintext=plaintext.lower()

plaintext 

'#indvpak kattpa #jadeja   ne bahubali #pandya  ki mar diys@tmsproducer the game was lost off the no ball wicket when bumrah was bowling #indvpak #ct2017final #ct17final #ct17#murshid \n#indvpak  #ct17 #ct17final https://t.co/gy3kvsfxii@chintskap    agli baar khoko ki team na bhejna\n\n#indvpakthis one was actually needed to be retweeted😛 #indvpak https://t.co/qyhqv6pwqlrt @abhishekprtp: lmao. indians leave the stadium. truly deserve it after all that abuse and vile jokes they\'ve been giving to pakistanis.…@chintskap are you there? \noh i forgot~you had to blue us . the phrase i remember is;\n"barking dog seldom bite"\n#getwellsoonrishi #indvpak 😎rt @hvgoenka: 11 indians stranded in oval, london. madam sushma swaraj please rescue them. #indvpakrt @saj_pakpassion: indian fans leaving the oval very quickly #ct17 #indvpak https://t.co/qdjsymcfrkswitching from one comedy show to the next #indvpak #kapilsharmasirf 156 abhe tak. i bet you can\'t even cross 180 #indvpak it\'s just not your d

In [24]:
char_list=sorted(list(set(plaintext)))
print(char_list)
plaintext=plaintext[:100000]

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '~', '\xa0', '¢', '£', '¥', '©', '®', '°', 'à', 'á', 'ê', 'í', 'ó', 'ö', 'ù', 'ü', 'ğ', 'ı', 'ş', 'ʖ', '̇', '͒', '͜', '͡', '،', '؟', 'ء', 'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ـ', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ى', 'ي', 'ٌ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٰ', 'ٹ', 'پ', 'چ', 'ڈ', 'ڑ', 'ک', 'گ', 'ں', 'ھ', 'ہ', 'ی', 'ے', 'ۓ', '۔', '۶', '۷', 'ँ', 'ं', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ऑ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'ङ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'ळ', 'व', 'श', 'ष', 'स', 'ह', '़',

In [25]:
char_to_num=dict((char,i) for i,char in enumerate(char_list))
num_to_char=dict((i,char) for i,char in enumerate(char_list))



In [26]:
seq_length=10
features=[]
labels=[]
skip_length=10
for i in range(0,len(plaintext)-seq_length,skip_length):
    X_train=plaintext[i:i+seq_length]
    Y_train=plaintext[i+seq_length]
    features.append([char_to_num[char] for char in X_train])
    labels.append([char_to_num[char] for char in Y_train])

labels=np.array(labels)
features=np.array(features)
print(np.shape(labels),np.shape(features))
print(labels[10,0])
def one_hot_encode_lists(output,length):
    a=[0 for i in range(length)]
    a[output]=1
    return np.array(a)
#print(features)
labels=np.array([one_hot_encode_lists(labels[i,0],len(char_list)) for i in range(len(labels))] )
features=np.array([[one_hot_encode_lists(feature[i],len(char_list)) for i in range(len(feature))] for feature in features])
print(features[0])

(9999, 1) (9999, 10)
56
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 1 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [27]:
print(features.shape)
   



(9999, 10, 904)


In [28]:
#X = np.reshape(features, (len(features), len(char_list),1))
X=features
print(X.shape)
print(X)
print(labels.shape)

(9999, 10, 904)
[[[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 ..., 
 [[0 1 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 1 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 1 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]

 [[0 0 0 ..., 0 0 0]
  [0 1 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  ..., 
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]
  [0 0 0 ..., 0 0 0]]]
(9999, 904)


In [30]:


model=Sequential()
model.add(LSTM(256,input_shape=[seq_length,len(char_list)],return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(units=labels.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam')
filepath="weights-improvement-tweet-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X,labels, epochs=10, batch_size=128, callbacks=callbacks_list)      


Epoch 1/10
9999/9999 [==============================] - 46s - loss: 4.0824    
Epoch 2/10
9999/9999 [==============================] - 47s - loss: 3.5857    
Epoch 3/10
9999/9999 [==============================] - 48s - loss: 3.5774    
Epoch 4/10
9999/9999 [==============================] - 49s - loss: 3.5688    
Epoch 5/10
9999/9999 [==============================] - 49s - loss: 3.5681    
Epoch 6/10
9999/9999 [==============================] - 51s - loss: 3.5551    
Epoch 7/10
9999/9999 [==============================] - 50s - loss: 3.5417    
Epoch 8/10
9999/9999 [==============================] - 51s - loss: 3.5091    
Epoch 9/10
9999/9999 [==============================] - 51s - loss: 3.5193    
Epoch 10/10
9999/9999 [==============================] - 51s - loss: 3.3540    


In [33]:
model.load_weights('weights-improvement-tweet-09-3.3540.hdf5')
seed=\n#indvpak#askcaptain #ct17 how much money  @imvkohli @ashwinravi99 &amp; @jaspritbumrah93  took to loose final?  #indvpak @10ankit1993 @si"
seed=seed[:10].lower()
for j in range(0,1000): 
    
    prediction=np.array([one_hot_encode_lists(char_to_num[char],len(char_list)) for char in seed])
  
    prediction=np.array([prediction])
    a=num_to_char[np.argmax(model.predict([prediction]))]
    
    print(a,end="")
    seed=seed+a
    
    seed=seed[1:]
    

 n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  n  